In [1]:
# Useful when running in a Jupyter notebook. Does not work in VS Code or with nbconvert
#%load_ext lab_black
#%config IPCompleter.greedy=True

In [2]:
import datetime as dt

import altair as alt
import altair_morberg.core as morberg
import pandas as pd
from IPython.display import Markdown as md

alt.themes.register("morberg_theme", morberg.theme)
alt.themes.enable("morberg_theme")

max_annual_distance = 15_000_000
csv_file = "odometer.csv"
# Dates to show in graph
start_date = "2020-06-01"
end_date = dt.datetime.today().isoformat()
days_driven = (dt.datetime.today() - pd.to_datetime(start_date)).days
target_distance = (
    max_annual_distance
    * (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
    / 365
)

max_distance_today = max_annual_distance * days_driven / 365

In [3]:
df_csv = pd.read_csv(csv_file, parse_dates=["date"], na_values=0).dropna()
df_csv = df_csv.melt("date", value_name="distance")
df_max = pd.DataFrame(
    {
        "date": pd.to_datetime([start_date, end_date], utc=True),
        "variable": "Max km",
        "distance": [0, target_distance],
    }
)
df = pd.concat([df_max, df_csv])
df.distance = df.distance / 1000

In [4]:
current = float(df.tail(1)["distance"])
remaining = max_distance_today / 1000 - current
current_avg = current / days_driven
max_avg = max_annual_distance / 1000 / 365
prognosis = current_avg * 365

In [6]:
base = alt.Chart(
    df,
    title=(
        {
            "text": "Körda kilometer sedan 1 juni 2020",
            "subtitle": f'I dag, {dt.datetime.today().strftime("%Y-%m-%d")}, har vi {remaining:,.0f} km kvar att köra till max utrymme.',
        }
    ),
    width=600,
    height=500,
).encode(
    x=alt.X(
        "yearmonthdate(date)",
        title=None,
        scale=alt.Scale(domain=(start_date, end_date)),
        axis=alt.Axis(format="%b %Y"),
    ),
    y=alt.Y("distance", axis=None),
    color=alt.Color("variable", title="", scale=alt.Scale(scheme="dark2"), legend=None),
    tooltip=[alt.Tooltip("date", title="Datum"), "variable"],
)

current_km = base.mark_line(size=4).transform_filter(alt.datum.variable == "odometer")
max_km = base.mark_line(size=1).transform_filter(alt.datum.variable == "Max km")

# See https://github.com/altair-viz/altair/issues/2405#issuecomment-778812971
labels = (
    alt.Chart(df)
    .mark_text(align="left", dx=3, fontSize=14)
    .encode(
        alt.X("date", aggregate="max"),
        alt.Y("distance", aggregate={"argmax": "date"}),
        alt.Text("label:N", aggregate={"argmax": "date"}),
        detail=alt.Detail("variable"),
    )
    .transform_calculate(
        label="(datum.variable == 'odometer' ? 'Har kört ' + round(datum.distance) + ' km' : 'Max ' + round(datum.distance) + ' km')"
    )
)

(labels + current_km + max_km).configure_axis(tickCount=4)

alt.LayerChart(...)